<a href="https://colab.research.google.com/github/digitalepidemiologylab/covid-bert/blob/master/Create_Twitter_Evaluation_Datasets_from_Sheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Create Vaccine Datasets from Sheets
Creates the vaccine datasets both for the annotated and the unannotated stuff stored in the Google Sheets.

In the end it copies the training datasets to the Google Cloud.

In [32]:
##@markdown ##Read Sheets
!pip install pandas_ml

import pandas as pd
import numpy as np
import gspread
import sys, os
from oauth2client.client import GoogleCredentials
from google.colab import auth
import pandas as pd

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

sheets = ['vaccine_sentiment_epfl','maternal_vaccine_stance_lshtm','twitter_sentiment_semeval']
data = {}
sh = gc.open('Twitter Evaluation Datasets')

for s in sheets:
  #Read the vaccine_sentiment_epfl
  worksheet = sh.worksheet(s)
  rows = worksheet.get_all_values()

  #Get it into pandas
  data[s] = pd.DataFrame.from_records(rows)
  data[s].columns = data[s].iloc[0]
  data[s]  = data[s].reindex(data[s].index.drop(0))
  data[s]['a'] = "a"

print("Successfully read the data")



Successfully read the annotated data


In [0]:
# #@markdown ##Create and Copy Data Files
import warnings
warnings.filterwarnings('ignore')

!rm -rf /content/datasets*

for s in sheets:
  if not os.path.exists('/content/datasets'):
    os.makedirs('/content/datasets')

  if not os.path.exists('/content/datasets/'+s):
    os.makedirs('/content/datasets/'+s)

  train = data[s][data[s]['dataset'] == "train"]
  dev = data[s][data[s]['dataset'] == "dev"]
  test = data[s][data[s]['dataset'] == "test"]


  with open('/content/datasets/'+s+"/train.tsv", 'w') as f:
    f.write(train.to_csv(columns=["id", "label", "a", "text"], header=False, index=False, sep='\t',encoding = 'utf-8'))

  with open('/content/datasets/'+s+"/dev.tsv", 'w') as f:
    f.write(dev.to_csv(columns=["id", "label", "a", "text"], header=False, index=False, sep='\t',encoding = 'utf-8'))

  #with open('/content/datasets/'+s+"/test.tsv", 'w') as f:
  #  f.write(test.to_csv(columns=["id", lang], header=False, index=False, sep='\t',encoding = 'utf-8'))

  with open('/content/datasets/'+s+"/test.tsv", 'w') as f:
    f.write(test.to_csv(columns=["id", "label", "a", "text"], header=False, index=False, sep='\t',encoding = 'utf-8'))


print ("We should now copy this to the bucket")

